In [966]:
import pandas as pd
import numpy as np
import networkx as nx
import random
import plotly.graph_objects as go

In [967]:
class Team():
    def __init__(self, name, rank):
        self.name = name
        self.rank = rank
        self.expectedResult = None
        self.finalResult = None
    
    def __repr__(self):
        return f"Team {self.name}"

class Game():
    def __init__(self, team1, team2):
        self.team1 = team1
        self.team2 = team2
        self.result = self.GetWinnerAndLoser()

    def GetWinnerAndLoser(self):
        if self.team1.rank < self.team2.rank:
            return {"Winner": self.team1, "Loser": self.team2}
        else:
            return {"Winner": self.team2, "Loser": self.team1}
    
    def __repr__(self):
        return str(self.result)

class Tournament():
    def __init__(self, teams):
        self.teams = teams
        self.result = None
        self.rounds = int(np.log2(len(self.teams)))

        # random initial bracket
        self.order = random.sample(self.teams, len(self.teams))
    
    def RunSingleElimination(self, displayResult = False):
        order = self.order.copy()
        level = {_: [] for _ in range(self.rounds+1)}

        for r in range(self.rounds):
            next_level = []

            # For each pair of teams, simulate a game between them
            for i in range(0, len(order), 2):
                G = Game(order[i], order[i + 1])

                # Add the loser to the list of losers this round
                level[r].append(G.result['Loser'])

                # next_round is the list of teams that advance to the next round
                next_level.append(G.result['Winner'])
            order = next_level.copy()

        # After all the rounds are done, the last team left is the winner
        level[self.rounds] = order.copy()
        self.result = level
        if displayResult:
            print(self.result)

    def CheckAccuracy(self, k):
        topRounds = int(np.log2(k))

        # Get the teams' ranking of a perfectly accurate result, grouped by which round they sohuld have advanced to
        correctResult = [set([0])] + [set(range(2**i,2**(i+1) )) for i in range(topRounds)]

        # Get the true rankings of teams, grouped by which round they advanced to
        actualResult = [set([t.rank for t in self.result[self.rounds-i]]) for i in range(topRounds+1)]

        return correctResult == actualResult
    
    def RunDoubleElimination(self, displayResult = False):
        self.RunSingleElimination()
        level = self.result.copy()

        # Run 2nd Knockout Bracket
        result = {_: [] for _ in range(2*self.rounds)}
        additionalRounds = 0
        for r in range(len(level.keys())):
            if r < len(level.keys()) -1:
                nextRoundTeams = len(level[r+1])
            else:
                nextRoundTeams = 0
            temp = []
            for i in range(0, len(level[r]), 2):
                G = Game(level[r][i], level[r][i+1])
                result[r + additionalRounds].append(G.result['Loser'])
                if int(len(level[r])/2) == nextRoundTeams:
                    level[r+1].insert(i, G.result['Winner'])
                else:
                    temp.append(G.result['Winner'])
            if len(temp) > 1:
                additionalRounds += 1
                for i in range(0, len(temp), 2):
                    G = Game(temp[i], temp[i+1])
                    level[r+1].insert(i, G.result['Winner'])
                    result[r+additionalRounds].append(G.result['Loser'])
        result[2*self.rounds-1] = temp.copy()
        self.result = result

        if displayResult:
            print(result)
    
    def convertResultToSingleElim(self):
        levels = len(self.result)
        singleResult = {int(i/2): self.result[i] + self.result[i+1] for i in range(0, levels-2, 2)}
        singleResult[len(singleResult)] = self.result[len(self.result) - 2]
        singleResult[len(singleResult)] = self.result[len(self.result) - 1]
        self.result = singleResult.copy()

    def CheckAccuracyDoubleElim(self, k):
        topRounds = int(np.log2(k))*2

        places = list(range(len(self.teams)))
        groupLengths = [len(self.result[k]) for k in self.result.keys()]
        groupLengths.reverse()
        expectedResult = []
        for i in groupLengths:
            expectedResult.append(set(places[0:i]))
            places = places[i:]
        correctResult = expectedResult[0:topRounds]
        actualResult = [set([t.rank for t in self.result[len(self.result.keys()) - 1 -i]]) for i in range(topRounds)]

        return correctResult == actualResult
    
    def DistanceToAccurateRanking(self, k):
        places = list(range(len(self.teams)))
        places.reverse()

        groupLengths = [len(self.result[k]) for k in self.result.keys()]

        expectedResult = dict()

        for i in range(len(self.result.keys())):
            numTeams = groupLengths[i]
            expectedResult[i] = (set(places[0:numTeams]))
            places = places[numTeams:]

        expectedResult = {rank:k for k in expectedResult.keys() for rank in expectedResult[k]}
        finalResult = {t:k for k in self.result.keys() for t in self.result[k]}

        for t in self.teams:
            t.expectedResult = expectedResult[t.rank]
            t.finalResult = finalResult[t]
        
        distance = sum([abs(t.expectedResult - t.finalResult) for t in self.teams if t.rank < k])
        return distance

In [968]:
def SimulateTournament(n, k, displayResult = False, useDistance = False):
    teams = []
    for i in range(n):
        teams.append(Team(i, i))
    T = Tournament(teams)
    T.RunSingleElimination(displayResult = displayResult)
    if useDistance:
        singleElim = T.DistanceToAccurateRanking(k)
    else:
        singleElim = T.CheckAccuracy(k)
    
    T.RunDoubleElimination(displayResult = displayResult)
    if useDistance:
        doubleElimStrong = T.DistanceToAccurateRanking(k)
    else:
        doubleElimStrong = T.CheckAccuracyDoubleElim(k)
    
    T.convertResultToSingleElim()
    if useDistance:
        doubleElimWeak = T.DistanceToAccurateRanking(k)
    else:
        doubleElimWeak = T.CheckAccuracy(k)
    return singleElim, doubleElimWeak, doubleElimStrong

In [969]:
def TestAccuracy(m, n, k, useDistance = False):
    results = {"SingleElimination": [], "DoubleEliminationWeak": [], "DoubleEliminationStrong": []}

    for _ in range(m):
        singleElim, doubleElimWeak, doubleElimStrong = SimulateTournament(n, k, False, useDistance=useDistance)
        results["SingleElimination"].append(singleElim)
        results["DoubleEliminationWeak"].append(doubleElimWeak)
        results["DoubleEliminationStrong"].append(doubleElimStrong)
    
    if useDistance:
        results = {key: [sum(results[key]) / len(results[key]) / k] for key in results.keys()}
    else:
        results = {key: [results[key].count(True) / len(results[key])] for key in results.keys()}
    results["n"] = [n]
    results["k"] = [k]

    return results


### Test Success Rate for Top k

In [970]:
m = 2000
test_values = {8: [2, 4], 16: [2, 4, 8], 32: [2, 4, 8], 64: [2, 4, 8, 16], 128: [2, 4, 8, 16, 32]}

results = {"n": [], "k": [], "SingleElimination": [], "DoubleEliminationWeak": [], "DoubleEliminationStrong": []}

for n in test_values.keys():
    for k in test_values[n]:
        test_output = TestAccuracy(m, n, k)
        results = {key: results[key] + test_output[key] for key in results.keys()}

df = pd.DataFrame(results)
df.round(3)

,n,k,SingleElimination,DoubleEliminationWeak,DoubleEliminationStrong
0,8,2,0.571,1.000,1.000
1,8,4,0.152,0.518,0.518
2,16,2,0.536,1.000,1.000
3,16,4,0.099,0.425,0.425
4,16,8,0.002,0.046,0.029
5,32,2,0.503,1.000,1.000
6,32,4,0.076,0.394,0.394
7,32,8,0.000,0.028,0.018
8,64,2,0.514,1.000,1.000
9,64,4,0.069,0.400,0.400


In [971]:
k_values = (4, 8) 
cols = ["SingleElimination", "DoubleEliminationWeak", "DoubleEliminationStrong"]
fig = go.Figure()
for k in k_values:
    slice = df[df["k"] == k]
    for col in cols:
        fig.add_scatter(name = f"k = {k}, {col}", x = slice["n"], y = slice[col], mode = "lines+markers")
fig.update_layout(title = "Percentage of Simulations where Top k is Correct", xaxis_title = "n", yaxis_title = "Frequency of Perfect Top k Final Ranking")
fig.show()

In [972]:
n_values = (16, 64) 
cols = ["SingleElimination", "DoubleEliminationWeak", "DoubleEliminationStrong"]
fig = go.Figure()
for n in n_values:
    slice = df[df["n"] == n]
    slice = slice[slice["k"] > 2]
    for col in cols:
        fig.add_scatter(name = f"n = {n}, {col}", x = slice["k"], y = slice[col], mode = "lines+markers")
fig.update_layout(title = "Percentage of Simulations where Top k is Correct", xaxis_title = "k", yaxis_title = "Frequency of Perfect Top k Final Ranking")
fig.show()

### Distance Metric

In [973]:
distance_results = {"n": [], "k": [], "SingleElimination": [], "DoubleEliminationWeak": [], "DoubleEliminationStrong": []}

for n in test_values.keys():
    for k in test_values[n]:
        test_output = TestAccuracy(m, n, k, True)
        distance_results = {key: distance_results[key] + test_output[key] for key in distance_results.keys()}

df = pd.DataFrame(distance_results)
df.round(3)

,n,k,SingleElimination,DoubleEliminationWeak,DoubleEliminationStrong
0,8,2,0.305,0.000,0.000
1,8,4,0.420,0.134,0.210
2,16,2,0.370,0.000,0.000
3,16,4,0.594,0.190,0.308
4,16,8,0.601,0.318,0.572
5,32,2,0.428,0.000,0.000
6,32,4,0.688,0.220,0.366
7,32,8,0.793,0.414,0.743
8,64,2,0.451,0.000,0.000
9,64,4,0.780,0.243,0.408


In [974]:
k_values = (4, 8) 
cols = ["SingleElimination", "DoubleEliminationWeak", "DoubleEliminationStrong"]
fig = go.Figure()
for k in k_values:
    slice = df[df["k"] == k]
    for col in cols:
        fig.add_scatter(name = f"k = {k}, {col}", x = slice["n"], y = slice[col], mode = "lines+markers")
fig.update_layout(title = "Average Distance From Accurate Final Ranking", xaxis_title = "n", yaxis_title = "Distance Metric")
fig.show()

In [975]:
n_values = (16, 64) 
cols = ["SingleElimination", "DoubleEliminationWeak", "DoubleEliminationStrong"]
fig = go.Figure()
for n in n_values:
    slice = df[df["n"] == n]
    for col in cols:
        fig.add_scatter(name = f"n = {n}, {col}", x = slice["k"], y = slice[col], mode = "lines+markers")
fig.update_layout(title = "Average Distance From Accurate Final Ranking", xaxis_title = "k", yaxis_title = "Distance Metric")
fig.show()

In [976]:
### Sandbox Code
# n = 32
# k = 8

# teams = []

# for i in range(n):
#     teams.append(Team(i, i))

# T = Tournament(teams)
# T.RunDoubleElimination(displayResult = True)
# print(T.CheckAccuracyDoubleElim(k))
# T.DistanceToAccurateRanking(k)

# topRounds = int(np.log2(k))*2

# places = list(range(len(T.teams)))
# places.reverse()

# groupLengths = [len(T.result[k]) for k in T.result.keys()]

# expectedResult = dict()

# for i in range(len(T.result.keys())):
#     numTeams = groupLengths[i]
#     expectedResult[i] = (set(places[0:numTeams]))
#     places = places[numTeams:]

# expectedResult = {rank:k for k in expectedResult.keys() for rank in expectedResult[k]}
# finalResult = {t:k for k in T.result.keys() for t in T.result[k]}
# finalResult